# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig

from azureml.train.hyperdrive.parameter_expressions import normal, choice

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import joblib

from train import clean_data, get_dataset

from sklearn.model_selection import train_test_split
import pandas as pd


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Detailed notes provided in the project file automl.ipynb

In [ ]:
ds = TabularDatasetFactory.from_delimited_files(https://github.com/dntrply/nd00333-capstone/blob/master/dataset/Breast_cancer_data.csv)
df = ds.to_pandas_dataframe()
df

In [ ]:
df.describe()

In [ ]:
ws = Workspace.from_config()

experiment=Experiment(ws, 'experiment-capstone-hyper')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# Use https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py as a reference
#

# See if it already exists
# this ofcourse helps if you are reunning the notebook from teh start and do not
# need to recreate the compute cluster
try:
    compute_cluster = ComputeTarget(workspace=ws, name='COMPUTE-HYPER')
    # No exception thrown - Found it - use it below
except ComputeTargetException:
    # Did not find the compute target - will need to create one
    # Specify the compute cluster configuration first
    # See https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target
    # and after clicking on Dv2
    # see https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target

    # CPU cluster instance - from the Microsoft recommended options
    cc_cfg = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS3_v2',
                                                            max_nodes=4,
                                                            description='CPU Compute Cluster created programatically for HyperParameter model')

    # Next, create the cluster
    cc = ComputeTarget.create(workspace=ws, name='COMPUTE_HYPER', provisioning_configuration=cc_cfg)

# We have a compute cluster - either newly created - or created earlier
# We may wait for the create operation to complete
cc.wait_for_completion(show_output=True)

Workspace name: quick-starts-ws-161910
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-161910
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
# Note that when creatingt he dataset, the ouput is changed from a quality score (1 - 10)
# to a binary 1/0 where 1 is applied to all entried where the quality score is 7 or greater
# The intent here is to define a dataset where the label is "desired" (or not) and not a
# quality score

train_ds = get_dataset(ws)

In [ ]:

# Take a peek at the data by converting the same to a Pandas dataframe
train_df = train_ds.to_pandas_dataframe()
    
# print the data
train_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1.712604,-1.371514,0.213258,0.474873,-0.904997,-1.311690,-1.843864,0.525802,-0.385870,-1.225374,0.435336,0
1,0.172079,-0.577820,-0.613053,-1.043248,-0.401518,-0.900098,-1.585029,-1.279671,-0.584545,-0.962507,0.963524,0
2,0.764589,0.215874,3.105347,-1.003816,5.228296,0.511075,0.250349,0.258324,-0.716995,-0.612016,-1.311750,0
3,-0.183426,0.017450,0.047996,0.494589,0.101961,-0.194512,-0.643809,0.174737,0.408829,0.439455,0.394706,1
4,-0.657434,-0.081762,-1.274101,0.257999,0.101961,0.569873,1.120977,0.726410,0.276379,-0.173903,-1.230491,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3913,1.120095,-0.875455,-0.117266,0.198851,-1.408477,0.687471,-0.337913,-0.417056,-1.313019,-0.874884,0.394706,0
3914,-0.183426,0.315085,-1.274101,0.257999,-0.126893,1.275460,1.450404,0.545862,-0.120971,-0.436771,-0.905451,0
3915,-0.538932,-1.272302,-0.365159,-0.905237,-0.081122,-0.841299,-0.549688,-0.791525,-0.584545,-0.962507,-0.255373,0
3916,0.527585,-0.379397,0.626414,1.421235,0.193503,0.334678,0.932733,1.595711,1.203527,-0.524394,-0.661672,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The train.py script uses [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression) from scikit-learn. The task is to maximize accuracy. 

### The parameterts to the model are:
Inverse of regularization strength : --C<br/>
Max Iterations : --max_iter

A random combination of values for these parameters will be iterated over to determine the optimal values for these two hyper parameters.
<br/><br/>

[Bandit Policy](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py) defines an early termination policy based on slack criteria (slack-factor)
<br/><br/>
The primary metric is accuracy, inferred by scikit-learn LogisticRegression


In [ ]:
# BanditPolicy is a type of EarlyTerminationPolicy
# More info can be found at 
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

# from train.py - we know that the parameters are --C and --max_iter
# train.py uses the scikit-learn LogisticRegression model

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10),
    "--max_iter": choice(50, 100, 200, 300, 400),
})

# We need to specify/provide an environment
# We could create our own environment with corresponding yaml file
# We may also provide an environment 'out-of-the-box' from one of the many environments provided

curated_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

# SKLearn is deprecated - instead we specity the run configuration using ScriptRunConfig
# It is necessary to pass the directory, the python script, the compute cluster and the environment

#TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>


run_config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=cc,
    environment=curated_env,
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    run_config=run_config,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
)

In [ ]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d
Web View: https://ml.azure.com/runs/HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-161910/workspaces/quick-starts-ws-161910&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-10-24T22:59:19.092532][API][INFO]Experiment created<END>\n""<START>[2021-10-24T22:59:19.870244][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2021-10-24T22:59:20.137608][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d
Web View: https://ml.azure.com/runs/HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-161910/workspaces/quick-starts-ws-161910&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d',
 'target': 'CPU-CC-HYPER',
 'status': 'Completed',
 'startTimeUtc': '2021-10-24T22:59:18.766745Z',
 'endTimeUtc': '2021-10-24T23:11:45.764135Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '38a73202-3b2d-486d-b27f-0e5f14529610',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.8255102040816327',
  'best_child_run_id': 'HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d_4',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg161910.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_7fab12a2-ca1a-4c2f-8788-a03c9c07069d/azureml-logs/hyperdrive.txt?sv

In [ ]:

assert(hyperdrive_run.get_status() == "Completed")


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.


### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the model hyperparameter values corresponding to the best run
print(f'********** {best_hyperdrive_run.get_details()["runDefinition"]["arguments"]}')
# print(best_hyperdrive_run.get_details())

# Get the accuracy corresponsding to the best run
print(f'********** Best HyperDrive run Accuracy: {best_hyperdrive_run.get_metrics()["Accuracy"]}')

********** ['--C', '1', '--max_iter', '50']
********** Best HyperDrive run Accuracy: 0.8255102040816327


In [ ]:
# Create the outputs directory
if 'outputs' not in os.listdir():
    os.mkdir('outputs'))

In [ ]:
#TODO: Save the best model
# Save and download the model
print(f'Run files: {best_hyperdrive_run.get_file_names()}')

# download the file in the notebook context. In this case, under Users/**user**/outputs/
best_hyperdrive_run.download_file(name=best_hyperdrive_run.get_file_names()[-1], output_file_path=os.path.join('outputs', 'best_hyper.pkl'))

best_hyperdrive_run.download_files(prefix=outputs, output_directory=outputs, output_paths=None, batch_size=100, append_prefix=False)

Run files: ['logs/azureml/17_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/best_hyperdrive.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'user_logs/std_log.txt']


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_hyperdrive_run.register_model(model_name='breast-cancer-hyper', 
                                           model_path = constants.MODEL_PATH,
                                           description='Breast Cancer detection using Azure HyperParameter',
                                           tags={'Method of execution':'HyperParameter'},
                                           properties={'Accuracy':best_hyperdrive_run.get_metrics()["Accuracy"]})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

